# Regression

## Load data

In [23]:
import sys
import pandas as pd
import numpy as np

#!wget wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm" -O data.zip
#!unzip hw1_data.zip
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
data = pd.read_csv('./train.csv', encoding = 'big5')
print ("Data overview")
print (data)

Data overview
              日期  測站          測項     0     1     2     3     4     5     6  \
0       2014/1/1  豐原    AMB_TEMP    14    14    14    13    12    12    12   
1       2014/1/1  豐原         CH4   1.8   1.8   1.8   1.8   1.8   1.8   1.8   
2       2014/1/1  豐原          CO  0.51  0.41  0.39  0.37  0.35   0.3  0.37   
3       2014/1/1  豐原        NMHC   0.2  0.15  0.13  0.12  0.11  0.06   0.1   
4       2014/1/1  豐原          NO   0.9   0.6   0.5   1.7   1.8   1.5   1.9   
...          ...  ..         ...   ...   ...   ...   ...   ...   ...   ...   
4315  2014/12/20  豐原         THC   1.8   1.8   1.8   1.8   1.8   1.7   1.7   
4316  2014/12/20  豐原       WD_HR    46    13    61    44    55    68    66   
4317  2014/12/20  豐原  WIND_DIREC    36    55    72   327    74    52    59   
4318  2014/12/20  豐原  WIND_SPEED   1.9   2.4   1.9   2.8   2.3   1.9   2.1   
4319  2014/12/20  豐原       WS_HR   0.7   0.8   1.8     1   1.9   1.7   2.1   

      ...    14    15    16    17    18    19    

## Preprocessing

In [24]:
# extract only number
data = data.iloc[:, 3:]
print (data)
# change the RAINFAIL value
data[data=='NR'] = 0
rawData = data.to_numpy()
print (rawData)

         0     1     2     3     4     5     6     7     8     9  ...    14  \
0       14    14    14    13    12    12    12    12    15    17  ...    22   
1      1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8   1.8  ...   1.8   
2     0.51  0.41  0.39  0.37  0.35   0.3  0.37  0.47  0.78  0.74  ...  0.37   
3      0.2  0.15  0.13  0.12  0.11  0.06   0.1  0.13  0.26  0.23  ...   0.1   
4      0.9   0.6   0.5   1.7   1.8   1.5   1.9   2.2   6.6   7.9  ...   2.5   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
4315   1.8   1.8   1.8   1.8   1.8   1.7   1.7   1.8   1.8   1.8  ...   1.8   
4316    46    13    61    44    55    68    66    70    66    85  ...    59   
4317    36    55    72   327    74    52    59    83   106   105  ...    18   
4318   1.9   2.4   1.9   2.8   2.3   1.9   2.1   3.7   2.8   3.8  ...   2.3   
4319   0.7   0.8   1.8     1   1.9   1.7   2.1     2     2   1.7  ...   1.3   

        15    16    17    18    19    20    21    2

## Extract Features

In [25]:
monthData = {}
for month in range(12):
    # 18 features * 480 hours
    sample = np.empty([18, 480])
    # 20 days value for a months
    for day in range(20):
        sample[:, day*24:(day+1)*24] = rawData[(month*20+day)*18:(month*20+day+1)*18,:]
    monthData[month] = sample
print (monthData)

{0: array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
       [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
       [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
       ...,
       [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
       [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
       [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]]), 1: array([[ 15.  ,  14.  ,  14.  , ...,   8.4 ,   8.  ,   7.6 ],
       [  1.8 ,   1.8 ,   1.7 , ...,   1.7 ,   1.7 ,   1.7 ],
       [  0.27,   0.26,   0.25, ...,   0.36,   0.35,   0.32],
       ...,
       [113.  , 109.  , 104.  , ...,  72.  ,  65.  ,  69.  ],
       [  2.3 ,   2.2 ,   2.6 , ...,   1.9 ,   2.9 ,   1.5 ],
       [  2.5 ,   2.2 ,   2.2 , ...,   0.9 ,   1.6 ,   1.1 ]]), 2: array([[ 18.  ,  18.  ,  18.  , ...,  14.  ,  13.  ,  13.  ],
       [  1.8 ,   1.8 ,   1.8 , ...,   1.8 ,   1.8 ,   1.8 ],
       [  0.39,   0.36,   0.4 , ...,   0.42,   0.47,   0.49],
       ...,
       [103.  , 128.  , 115.  , ...,  60.  ,  94.  ,  53

In [26]:
# 9 hours for one data
x = np.empty([471*12, 9*18], dtype = float)
y = np.empty([471*12, 1], dtype = float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                # the last data in one day is 14~24
                continue
            x[month*471+day*24+hour, :] = monthData[month][:, day*24+hour:day*24+hour+9].reshape(1,-1)
            y[month*471+day*24+hour, 0] = monthData[month][9, day*24+hour+9]
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


## Normalize

In [27]:
xMean = np.mean(x, axis = 0)
xStd = np.std(x, axis = 0)

for i in range(len(x)):
    for j in range(len(x[i])):
        if xStd[j] != 0:
            x[i][j] = (x[i][j] - xMean[j])/xStd[j]
print(x)

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]


## Split training data into TrainSet and ValidationSet

In [86]:
import math
xTrain = x[:math.floor(len(x)*0.8), :]
yTrain = y[:math.floor(len(y)*0.8), :]
xValid = x[math.floor(len(x)*0.8):, :]
yValid = y[math.floor(len(y)*0.8):, :]

## Train

In [87]:

dim = 9*18 + 9*18 + 1
# weight
w = np.zeros([dim,1])
# bias + x + x^2
xTrain_2 = np.power(xTrain, 2)
xTrain = np.concatenate((np.ones([math.floor(12*471*0.8), 1]), xTrain), axis = 1).astype(float)
xTrain = np.concatenate((xTrain, xTrain_2), axis = 1).astype(float)
learningRate = 100
iterTime = 100000

'''
# bias + x
dim = 9*18 + 1
# weight
w = np.zeros([dim,1])
xTrain = np.concatenate((np.ones([math.floor(12*471*0.8), 1]), xTrain), axis = 1).astype(float)
learningRate = 100
iterTime = 100000
'''
adagrad = np.zeros([dim, 1])
eps = 0.00000000001
for i in range(iterTime):
    loss = np.sqrt(np.sum(np.power(np.dot(xTrain, w) - yTrain, 2))/471/12) # RMSE
    if i%1000 == 0:
        print (loss)
    gradient = 2*np.dot(xTrain.transpose(), np.dot(xTrain, w) - yTrain)
    adagrad += gradient ** 2
    w = w - learningRate * gradient/np.sqrt(adagrad + eps)

24.36221492236629
8.612220823464387
6.278730315452408
5.6134139363942746
5.2934042153508365
5.132619870471894
5.050063183633019
5.006667257446449
4.983247317713033
4.97023589143872
4.962774255222113
4.958344897148382
4.955615304737339
4.953864335566986
4.952692615111012
4.951873639662502
4.9512757755516486
4.950820605359273
4.950460239812543
4.950164705662889
4.949914786657191
4.949697872536971
4.949505500185713
4.949331865931007
4.949172906584358
4.949025720316558
4.948888194640077
4.9487587630430205
4.94863624301358
4.948519726456759
4.948408504394389
4.948302014448955
4.948199803692722
4.948101502005043
4.94800680271291
4.947915448345125
4.947827220021954
4.947741929460919
4.947659412887497
4.9475795263488
4.947502142071955
4.94742714560868
4.947354433577453
4.94728391186427
4.947215494178455
4.9471491008856265
4.947084658058715
4.947022096701709
4.9469613521111375
4.94690236334802
4.946845072798912
4.946789425809155
4.946735370374905
4.94668285688323
4.9466318378916485
4.9465822679

## Validation

In [88]:
xValid_2 = np.power(xValid, 2)
xValid = np.concatenate((np.ones([math.ceil(12*471*0.2), 1]), xValid), axis = 1).astype(float)
xValid = np.concatenate((xValid, xValid_2), axis = 1).astype(float)
loss = np.sqrt(np.sum(np.power(np.dot(xValid, w) - yValid, 2))/471/12)
print (loss)

2.5469743298095184


## Train by All Data

In [89]:
dim = 9*18 + 9*18 + 1
# weight
w = np.zeros([dim,1])
# bias + x + x^2
xTrain_2 = np.power(x, 2)
xTrain = np.concatenate((np.ones([12*471, 1]), x), axis = 1).astype(float)
xTrain = np.concatenate((xTrain, xTrain_2), axis = 1).astype(float)
learningRate = 100
iterTime = 100000

adagrad = np.zeros([dim, 1])
eps = 0.00000000001
for i in range(iterTime):
    loss = np.sqrt(np.sum(np.power(np.dot(xTrain, w) - y, 2))/471/12) # RMSE
    if i%1000 == 0:
        print (loss)
    gradient = 2*np.dot(xTrain.transpose(), np.dot(xTrain, w) - y)
    adagrad += gradient ** 2
    w = w - learningRate * gradient/np.sqrt(adagrad + eps)
np.save('weight.npy', w)
print (w)

27.071214829194115
15.473639974038262
8.466962882655814
6.781724211306865
6.18731877381183
5.9009126257380435
5.7454598531778815
5.65712612889171
5.605344438718864
5.574063156481417
5.554576253964487
5.542059096218049
5.533778785980463
5.528149415509408
5.524226248348203
5.521430852462861
5.519399157599226
5.517895805425834
5.5167648026110285
5.515900405591826
5.515229505173996
5.5147006914800505
5.514277291493953
5.513932827538969
5.513647988132494
5.513408566856883
5.513204035233888
5.513026539698551
5.512870187660892
5.5127305338940875
5.512604207715482
5.512488640312074
5.512381864021642
5.512282363758019
5.512188966497705
5.512100758720416
5.512017024490102
5.511937198847872
5.511860832611375
5.511787565703448
5.511717106880831
5.511649218280838
5.511583703606147
5.511520399064873
5.5114591664032355
5.511399887531916
5.511342460369432
5.511286795617344
5.51123281425083
5.511180445559861
5.511129625615253
5.511080296063425
5.511032403176114
5.5109858970983225
5.510940731250832
5.51

## Test

In [90]:
testData = pd.read_csv('./test.csv', header = None, encoding = 'big5')
testData = testData.iloc[:, 2:]
testData[testData=='NR'] = 0
testData = testData.to_numpy()
xTest = np.empty([240, 18*9], dtype = float)
for i in range(240):
    xTest[i,:] = testData[i*18:(i+1)*18,:].reshape(1, -1)
for i in range(len(xTest)):
    for j in range(len(xTest[i])):
        if xStd[j] != 0:
            xTest[i][j] = (xTest[i][j] - xMean[j])/xStd[j]
xTest_2 = np.power(xTest, 2)
xTest = np.concatenate((np.ones([240, 1]), xTest), axis = 1).astype(float)
xTest = np.concatenate((xTest, xTest_2), axis = 1).astype(float)
print (xTest)

[[ 1.00000000e+00 -2.44476813e-01 -2.45459187e-01 ...  4.49776666e-01
   1.09399870e+00  6.08071366e-03]
 [ 1.00000000e+00 -1.35825331e+00 -1.51789368e+00 ...  2.98567879e-02
   1.18962443e-02  2.34783137e-01]
 [ 1.00000000e+00  1.50574340e+00  1.34508393e+00 ...  1.76004466e+00
   1.09399870e+00  3.34428717e-01]
 ...
 [ 1.00000000e+00  3.91966901e-01  5.49812369e-01 ...  7.10261347e-02
   4.11105259e-02  1.44726989e+00]
 [ 1.00000000e+00 -1.83558610e+00 -1.83600230e+00 ...  1.09310870e+00
   1.29875957e+00  1.30147326e+00]
 [ 1.00000000e+00 -1.35825331e+00 -1.35883937e+00 ...  8.90589585e+00
   1.06515848e+01  3.11716148e+00]]


In [91]:
w = np.load('weight.npy')
ans_y = np.dot(xTest, w)
print (ans_y)

[[  8.96033047]
 [ 18.76178677]
 [ 24.67784686]
 [ 10.77687214]
 [ 27.57110815]
 [ 22.62854244]
 [ 21.86111579]
 [ 30.54991853]
 [ 15.90490687]
 [ 59.21928484]
 [ 11.98215934]
 [ 10.90329505]
 [ 63.42706219]
 [ 52.18867483]
 [ 22.64206831]
 [ 12.78148273]
 [ 31.20192399]
 [ 70.0159861 ]
 [  1.15779139]
 [ 16.8408867 ]
 [ 41.97373445]
 [ 75.5420776 ]
 [ 10.29010908]
 [ 17.02653977]
 [ 15.76022518]
 [ 39.21616409]
 [ 14.47940488]
 [ 62.28765297]
 [  6.54769102]
 [ 56.08009953]
 [ 20.92143137]
 [ 11.50244152]
 [  7.78335183]
 [ 20.82636916]
 [ 28.64463027]
 [ 37.73746399]
 [ 44.16479964]
 [ 31.17725281]
 [ 44.49318434]
 [ 37.26912516]
 [  8.37351445]
 [ 40.35783925]
 [ 31.95896478]
 [ 50.81791583]
 [ 18.48897579]
 [ 35.53394101]
 [ 25.44234634]
 [ 11.62563766]
 [ 25.93416495]
 [ 32.01757897]
 [ 23.38501446]
 [  7.34801844]
 [ 17.59299182]
 [ 53.09878458]
 [ 15.11001755]
 [ 36.07321767]
 [ 35.11410994]
 [ 20.045089  ]
 [ 57.74956583]
 [ 23.25157024]
 [ 16.38068356]
 [ 41.04858551]
 [ 11.96

In [92]:
import csv

with open('submit.csv', mode = 'w', newline = '') as submitFile:
    csvWriter = csv.writer(submitFile)
    header = ['id', 'value']
    print (header)
    csvWriter.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csvWriter.writerow(row)
        print (row)

['id', 'value']
['id_0', 8.960330472280063]
['id_1', 18.761786768270536]
['id_2', 24.677846857124514]
['id_3', 10.776872140873174]
['id_4', 27.57110815341475]
['id_5', 22.628542438281976]
['id_6', 21.861115790038948]
['id_7', 30.549918526231426]
['id_8', 15.904906868446963]
['id_9', 59.21928483557053]
['id_10', 11.982159344122243]
['id_11', 10.903295047035613]
['id_12', 63.42706219460371]
['id_13', 52.188674834064706]
['id_14', 22.642068308410376]
['id_15', 12.78148273117323]
['id_16', 31.201923987504177]
['id_17', 70.01598610163819]
['id_18', 1.1577913893654477]
['id_19', 16.84088670226007]
['id_20', 41.97373444865513]
['id_21', 75.54207759513781]
['id_22', 10.290109080717253]
['id_23', 17.026539770730526]
['id_24', 15.760225182406932]
['id_25', 39.21616408911181]
['id_26', 14.479404875663718]
['id_27', 62.287652968979366]
['id_28', 6.547691021813606]
['id_29', 56.08009953399227]
['id_30', 20.92143137136447]
['id_31', 11.502441519586634]
['id_32', 7.783351828381268]
['id_33', 20.82636